In [ ]:
#basic imports ✌

import numpy as np
import math, random
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(0)

In [ ]:
#creating the signals 👌

def sine(X,signal_freq=60.):
    return np.sin(2*np.pi*(X)/signal_freq)

def noisy(Y,noise_range=(-0.35,0.35)):
    noise=np.random.uniform(noise_range[0],noise_range[1],size=Y.shape)
    return Y+noise

def sample(sample_size):
    random_offset = random.randint(0, sample_size)
    X = np.arange(sample_size)
    out = sine(X + random_offset)
    inp = noisy(out)
    return inp, out

In [ ]:
# generating figures 👍👍👍👍👍

inp, out = sample(100)
plt.plot(inp, label='Noisy')
plt.plot(out, label ='Denoised')
plt.legend()

In [ ]:
#genereate dataset 👀👀👀

def create_dataset(n_samples=10000, sample_size=100):
    data_inp= np.zeros((n_samples,sample_size))
    data_out= np.zeros((n_samples,sample_size))
    
    for i in range(n_samples):
        sample_inp, sample_out = sample(sample_size)
        data_inp[i, :] = sample_inp
        data_out[i, :] = sample_out
    return data_inp,data_out

In [ ]:
data_inp,data_out = create_dataset()
train_inp, train_out= data_inp[:8000], data_out[:8000]
test_inp, test_out = data_inp[8000:], data_out[8000:]

In [ ]:
import torch 
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
#The RNN 👌👌👌👌👌

input_dim=1
hidden_size=30
num_layers=1

class CustomRNN(nn.Module):
    def __init__(self, input_size, hiddden_size, output_size):
        super(CustomRNN, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size, )
        self.act= nn.Tanh()
    def forward(self, x):
        pred,hiddne= self.rnn(x,None)
        pred = self.act(self.linear(pred)).view(pred.data.shape[0], -1,1)
        return pred
    
r= CustomRNN(input_dim, hidden_size, 1)

In [ ]:
r

In [ ]:
# TRAINING 💪💪💪💪💪💪💪💪💪💪💪

predictions= []

optimizer = torch.optim.Adam(r.parameters(), lr=1e-2)
loss_func = nn.L1Loss()

for t in range(301):
    hidden= None
    inp = Variable(torch.Tensor(train_inp.reshape((train_inp.shape[0], -1, 1))), requires_grad=True)
    out=Variable(torch.Tensor(train_out.reshape((train_out.shape[0], -1, 1))) )
    pred= r(inp)
    optimizer.zero_grad()
    predictions.append(pred.data.numpy())
    loss= loss_func(pred, out)
    if t%20==0:
        print(t, loss.data[0])
    loss.backward()
    optimizer.step()
    